## 🛒 ระบบจัดการสินค้า ISP (Inventory & Logistics)
---

ในบทนี้เราจะจำลองข้อมูลที่มี 3 ส่วนหลัก (3 Sheets) เพื่อให้ป๋าฝึกการจัดการข้อมูลที่ซับซ้อนขึ้น:
1. **Orders (ใบสั่งซื้อ):** รายการที่ลูกค้าสั่งอุปกรณ์ (เช่น Router, Mesh WiFi)
2. **Deliveries (การส่งสินค้า):** สถานะการขนส่ง
3. **Claims (การเคลม):** รายการสินค้าที่มีปัญหาและส่งคืน

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Relational Data** | ริ-เล-ชัน-นอล ดาต้า | **ข้อมูลที่มีความสัมพันธ์กัน** (เช่น ใบเคลมต้องอ้างอิงถึงใบสั่งซื้อ) |
| **Transaction ID** | แทรน-แซก-ชัน ไอ-ดี | **รหัสรายการ** (ใช้เป็นตัวระบุหลักในแต่ละแถว) |
| **Logistics Status** | โล-จิส-ติกส์ สเต-ตัส | สถานะการขนส่ง (เช่น Pending, Shipped, Delivered) |
| **Cross-Reference** | ครอส-เรฟ-เฟอ-เรนซ์ | การตรวจสอบข้อมูลข้ามตาราง |

### 🔍 สิ่งที่ป๋าต้องทบทวน (Review)
* การใช้ `pd.ExcelFile` เพื่ออ่านหลาย Sheet ในไฟล์เดียว
* การจัดการค่าว่าง (`NaN`) ในตาราง Claims และ Deliveries

💻 ส่วนที่ 2: โค้ดสร้าง Mock Data "ระบบจัดการสินค้า" (Code Cell)
ป๋ารันโค้ดนี้เพื่อสร้างไฟล์ isp_supply_chain.xlsx ที่มีข้อมูลแสนแถว (จำลอง) และหลายคอลัมน์ครับ

In [1]:
import pandas as pd
import numpy as np

# 1. สร้างตาราง Orders (ใบสั่งซื้อ)
orders_data = {
    'ORDER_ID': [f'ORD{i:05d}' for i in range(1, 11)],
    'MSISDN': ['0960000001', '0960000002', '0960000003', '0960000004', '0960000005', 
               '0960000006', '0960000007', '0960000008', '0960000009', '0960000010'],
    'PRODUCT': ['Router WiFi 6', 'Mesh Node', 'Fiber Modem', 'CCTV Cloud', 'Router WiFi 6',
                'Mesh Node', 'Router WiFi 6', 'Fiber Modem', 'CCTV Cloud', 'Mesh Node'],
    'PRICE': [2900, 1500, 0, 990, 2900, 1500, 2900, 0, 990, 1500],
    'ORDER_DATE': pd.date_range(start='2026-01-01', periods=10, freq='D')
}
df_orders = pd.DataFrame(orders_data)

# 2. สร้างตาราง Deliveries (การส่งสินค้า - มีค่าว่างบางจุด)
deliveries_data = {
    'DELIVERY_ID': [f'DLV{i:05d}' for i in range(1, 11)],
    'ORDER_ID': [f'ORD{i:05d}' for i in range(1, 11)],
    'CARRIER': ['Kerry', 'Flash', 'J&T', 'Kerry', 'Flash', np.nan, 'J&T', 'Kerry', np.nan, 'Flash'],
    'STATUS': ['Delivered', 'Delivered', 'Shipped', 'Delivered', 'Pending', 'Cancelled', 'Delivered', 'Shipped', 'Pending', 'Delivered'],
    'DELIVERY_DATE': ['2026-01-05', '2026-01-06', np.nan, '2026-01-08', np.nan, np.nan, '2026-01-10', np.nan, np.nan, '2026-01-12']
}
df_deliveries = pd.DataFrame(deliveries_data)

# 3. สร้างตาราง Claims (การเคลม - มีเฉพาะบางรายการ)
claims_data = {
    'CLAIM_ID': ['CLM001', 'CLM002'],
    'ORDER_ID': ['ORD00001', 'ORD00007'], # เคลมเฉพาะออเดอร์ 1 และ 7
    'REASON': ['Power not on', 'Broken Box'],
    'CLAIM_STATUS': ['Approved', 'Pending']
}
df_claims = pd.DataFrame(claims_data)

# บันทึกเป็นไฟล์เดียวแต่มี 3 Sheets
with pd.ExcelWriter('../Lesson_3/isp_supply_chain.xlsx') as writer:
    df_orders.to_excel(writer, sheet_name='Orders', index=False)
    df_deliveries.to_excel(writer, sheet_name='Deliveries', index=False)
    df_claims.to_excel(writer, sheet_name='Claims', index=False)

print("ป๋าครับ! ไฟล์ระบบ Supply Chain (3 Sheets) พร้อมให้เรียนแล้วครับ")

ป๋าครับ! ไฟล์ระบบ Supply Chain (3 Sheets) พร้อมให้เรียนแล้วครับ


💻 ส่วนที่ 3: บททดสอบการอ่านและเช็กสุขภาพข้อมูล (Code Cell)
ป๋าลองใช้ความรู้จาก บทที่ 1 และ บทที่ 4 มาประยุกต์ใช้กับข้อมูลชุดใหม่นี้ครับ

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# 1. ใช้ ExcelFile เพื่อเปิดไฟล์ (ทบทวนบทที่ 1)
xls = pd.ExcelFile('isp_supply_chain.xlsx')

# 2. อ่านทั้ง 3 ชีทออกมา
orders = xls.parse('Orders')
deliveries = xls.parse('Deliveries')
claims = xls.parse('Claims')

# 3. เช็กว่าในตาราง Deliveries มี 'รูรั่ว' (ค่าว่าง) ตรงไหนบ้าง (ทบทวนบทที่ 4)
print("ตรวจสอบค่าว่างในตารางการจัดส่ง:")
print(deliveries.isna().sum())

# 4. ลองดูว่าออเดอร์ไหนบ้างที่มีการเคลม (ทบทวนการมองหาความสัมพันธ์)
display(claims.head())

ตรวจสอบค่าว่างในตารางการจัดส่ง:
DELIVERY_ID      0
ORDER_ID         0
CARRIER          2
STATUS           0
DELIVERY_DATE    5
dtype: int64


,CLAIM_ID,ORDER_ID,REASON,CLAIM_STATUS
0,CLM001,ORD00001,Power not on,Approved
1,CLM002,ORD00007,Broken Box,Pending


In [3]:
print('orders info')
print(orders.info())
display(orders)
orders['PRICE'] = orders['PRICE'].apply(float)
orders['ORDER_DATE'] = orders['ORDER_DATE'].apply(pd.to_datetime)

orders info
<class 'pandas.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ORDER_ID    10 non-null     str           
 1   MSISDN      10 non-null     int64         
 2   PRODUCT     10 non-null     str           
 3   PRICE       10 non-null     int64         
 4   ORDER_DATE  10 non-null     datetime64[us]
dtypes: datetime64[us](1), int64(2), str(2)
memory usage: 532.0 bytes
None


,ORDER_ID,MSISDN,PRODUCT,PRICE,ORDER_DATE
0,ORD00001,960000001,Router WiFi 6,2900,2026-01-01
1,ORD00002,960000002,Mesh Node,1500,2026-01-02
2,ORD00003,960000003,Fiber Modem,0,2026-01-03
3,ORD00004,960000004,CCTV Cloud,990,2026-01-04
4,ORD00005,960000005,Router WiFi 6,2900,2026-01-05
5,ORD00006,960000006,Mesh Node,1500,2026-01-06
6,ORD00007,960000007,Router WiFi 6,2900,2026-01-07
7,ORD00008,960000008,Fiber Modem,0,2026-01-08
8,ORD00009,960000009,CCTV Cloud,990,2026-01-09
9,ORD00010,960000010,Mesh Node,1500,2026-01-10


In [4]:
print('deliverry info')
print(deliveries.info())
deliveries['DELIVERY_DATE'] = pd.to_datetime(deliveries['DELIVERY_DATE'])
deliveries['CARRIER'] = deliveries['CARRIER'].fillna('Unknown Customer')
display(deliveries)



deliverry info
<class 'pandas.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   DELIVERY_ID    10 non-null     str  
 1   ORDER_ID       10 non-null     str  
 2   CARRIER        8 non-null      str  
 3   STATUS         10 non-null     str  
 4   DELIVERY_DATE  5 non-null      str  
dtypes: str(5)
memory usage: 532.0 bytes
None


,DELIVERY_ID,ORDER_ID,CARRIER,STATUS,DELIVERY_DATE
0,DLV00001,ORD00001,Kerry,Delivered,2026-01-05
1,DLV00002,ORD00002,Flash,Delivered,2026-01-06
2,DLV00003,ORD00003,J&T,Shipped,NaT
3,DLV00004,ORD00004,Kerry,Delivered,2026-01-08
4,DLV00005,ORD00005,Flash,Pending,NaT
5,DLV00006,ORD00006,Unknown Customer,Cancelled,NaT
6,DLV00007,ORD00007,J&T,Delivered,2026-01-10
7,DLV00008,ORD00008,Kerry,Shipped,NaT
8,DLV00009,ORD00009,Unknown Customer,Pending,NaT
9,DLV00010,ORD00010,Flash,Delivered,2026-01-12


# 🔗 บทที่ 6: การรวมร่างข้อมูล (Data Merging & Joining)
---

ในบทนี้ เราจะนำข้อมูลจากหลาย ๆ ตารางมาเชื่อมต่อกันโดยใช้ "ตัวเชื่อม" ที่เหมือนกัน (เช่น `ORDER_ID`) เพื่อให้เห็นภาพรวมตั้งแต่ต้นจนจบ (End-to-End)

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Merge / Join** | เมิร์จ / จอย | **การรวมตาราง** สองตารางเข้าด้วยกัน |
| **Key / On** | คีย์ / ออน | **คอลัมน์ตัวเชื่อม** (หัวข้อที่มีข้อมูลเหมือนกันทั้งสองตาราง) |
| **Inner Join** | อิน-เนอร์ จอย | รวมเฉพาะข้อมูลที่มี **คู่แมตช์กันทั้งสองฝั่ง** เท่านั้น |
| **Left Join** | เลฟต์ จอย | ยึดตารางซ้าย (หลัก) เป็นตัวตั้ง **ข้อมูลฝั่งซ้ายต้องอยู่ครบ** แม้ไม่มีคู่ฝั่งขวาก็ตาม |
| **Right Join** | ไรต์ จอย | ยึดตารางขวาเป็นตัวตั้ง (ไม่ค่อยนิยม เพราะเรามักเขียนสลับฝั่งเอาได้) |

### 🔍 สิ่งที่ป๋าต้องทบทวน (Review)
1. การอ่านหลาย Sheets ด้วย `pd.ExcelFile`
2. การใช้ `head()` เพื่อเช็กชื่อคอลัมน์ก่อนรวมร่าง (ถ้าชื่อ Key ไม่ตรงกันจะรวมไม่ได้!)

In [5]:
display(orders)
display(deliveries)
display(claims)

,ORDER_ID,MSISDN,PRODUCT,PRICE,ORDER_DATE
0,ORD00001,960000001,Router WiFi 6,2900.0,2026-01-01
1,ORD00002,960000002,Mesh Node,1500.0,2026-01-02
2,ORD00003,960000003,Fiber Modem,0.0,2026-01-03
3,ORD00004,960000004,CCTV Cloud,990.0,2026-01-04
4,ORD00005,960000005,Router WiFi 6,2900.0,2026-01-05
5,ORD00006,960000006,Mesh Node,1500.0,2026-01-06
6,ORD00007,960000007,Router WiFi 6,2900.0,2026-01-07
7,ORD00008,960000008,Fiber Modem,0.0,2026-01-08
8,ORD00009,960000009,CCTV Cloud,990.0,2026-01-09
9,ORD00010,960000010,Mesh Node,1500.0,2026-01-10


,DELIVERY_ID,ORDER_ID,CARRIER,STATUS,DELIVERY_DATE
0,DLV00001,ORD00001,Kerry,Delivered,2026-01-05
1,DLV00002,ORD00002,Flash,Delivered,2026-01-06
2,DLV00003,ORD00003,J&T,Shipped,NaT
3,DLV00004,ORD00004,Kerry,Delivered,2026-01-08
4,DLV00005,ORD00005,Flash,Pending,NaT
5,DLV00006,ORD00006,Unknown Customer,Cancelled,NaT
6,DLV00007,ORD00007,J&T,Delivered,2026-01-10
7,DLV00008,ORD00008,Kerry,Shipped,NaT
8,DLV00009,ORD00009,Unknown Customer,Pending,NaT
9,DLV00010,ORD00010,Flash,Delivered,2026-01-12


,CLAIM_ID,ORDER_ID,REASON,CLAIM_STATUS
0,CLM001,ORD00001,Power not on,Approved
1,CLM002,ORD00007,Broken Box,Pending


In [10]:
# 2. ทำการรวมร่าง (Merge) แบบ Left Join
# เราอยากรู้ว่า "ออเดอร์ทุกลูกค้า" (ฝั่งซ้าย) มีสถานะส่ง (ฝั่งขวา) เป็นยังไงบ้าง
# โดยใช้ 'ORDER_ID' เป็นตัวเชื่อม
df_merged = pd.merge(orders, deliveries, on='ORDER_ID', how='left')


# 2. รวมร่างต่อจาก df_merged ที่ทำไว้ตะกี้ (เป็น 3 ตารางรวมกัน)
# คราวนี้เราจะเห็นว่ารายการไหน 'มีการเคลม' บ้าง
df_final = pd.merge(df_merged, df_claims, on='ORDER_ID', how='left')

# 3. ทบทวนบทที่ 4: เติมค่าว่างในคอลัมน์เคลม
# ถ้าไม่มีข้อมูลเคลม ให้ขึ้นว่า 'No Claim'
df_final['CLAIM_STATUS'] = df_final['CLAIM_STATUS'].fillna('No Claim')

# 4. แสดงผลลัพธ์สุดท้าย
# ป๋าจะเห็นภาพตั้งแต่: ใครสั่งอะไร -> ส่งโดยใคร -> สถานะอะไร -> มีเคลมไหม
display(df_final[['ORDER_ID', 'PRODUCT', 'STATUS', 'CLAIM_STATUS']])

,ORDER_ID,PRODUCT,STATUS,CLAIM_STATUS
0,ORD00001,Router WiFi 6,Delivered,Approved
1,ORD00002,Mesh Node,Delivered,No Claim
2,ORD00003,Fiber Modem,Shipped,No Claim
3,ORD00004,CCTV Cloud,Delivered,No Claim
4,ORD00005,Router WiFi 6,Pending,No Claim
5,ORD00006,Mesh Node,Cancelled,No Claim
6,ORD00007,Router WiFi 6,Delivered,Pending
7,ORD00008,Fiber Modem,Shipped,No Claim
8,ORD00009,CCTV Cloud,Pending,No Claim
9,ORD00010,Mesh Node,Delivered,No Claim


## 🧐 การวิเคราะห์หา "จุดบกพร่อง" หลัง Merge
---

เมื่อป๋ารวมร่างแบบ **Left Join** แล้ว สิ่งที่ป๋าจะพบคือบางแถวอาจจะมีค่าเป็น `NaN` ในฝั่งของข้อมูลการส่ง (Deliveries) ซึ่งมันบอกอะไรป๋าได้หลายอย่างครับ:

| ผลลัพธ์ที่เจอ | ความหมายในเชิงระบบ ISP |
| :--- | :--- |
| **มีข้อมูลครบทั้งสองฝั่ง** | การสั่งซื้อปกติ และเข้าระบบส่งเรียบร้อย |
| **ฝั่ง Deliveries เป็น NaN** | ลูกค้าสั่งซื้อแล้ว แต่ยังไม่มีข้อมูลในระบบขนส่ง (อาจจะหลุด หรือยังไม่แพ็ก) |

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Mismatch** | มิส-แมตช์ | ข้อมูลไม่ตรงกัน หรือหาคู่ไม่เจอ |
| **Data Integrity** | ดาต้า อิน-เท-กริ-ตี | ความถูกต้องและสมบูรณ์ของข้อมูล |

In [11]:
# 2. รวมร่างต่อจาก df_merged ที่ทำไว้ตะกี้ (เป็น 3 ตารางรวมกัน)
# คราวนี้เราจะเห็นว่ารายการไหน 'มีการเคลม' บ้าง
df_final = pd.merge(df_merged, df_claims, on='ORDER_ID', how='left')

# 3. ทบทวนบทที่ 4: เติมค่าว่างในคอลัมน์เคลม
# ถ้าไม่มีข้อมูลเคลม ให้ขึ้นว่า 'No Claim'
df_final['CLAIM_STATUS'] = df_final['CLAIM_STATUS'].fillna('No Claim')

# 4. แสดงผลลัพธ์สุดท้าย
# ป๋าจะเห็นภาพตั้งแต่: ใครสั่งอะไร -> ส่งโดยใคร -> สถานะอะไร -> มีเคลมไหม
display(df_final[['ORDER_ID', 'PRODUCT', 'STATUS', 'CLAIM_STATUS']])

,ORDER_ID,PRODUCT,STATUS,CLAIM_STATUS
0,ORD00001,Router WiFi 6,Delivered,Approved
1,ORD00002,Mesh Node,Delivered,No Claim
2,ORD00003,Fiber Modem,Shipped,No Claim
3,ORD00004,CCTV Cloud,Delivered,No Claim
4,ORD00005,Router WiFi 6,Pending,No Claim
5,ORD00006,Mesh Node,Cancelled,No Claim
6,ORD00007,Router WiFi 6,Delivered,Pending
7,ORD00008,Fiber Modem,Shipped,No Claim
8,ORD00009,CCTV Cloud,Pending,No Claim
9,ORD00010,Mesh Node,Delivered,No Claim


# 📊 บทสรุป: รายงานวิเคราะห์การเคลมสินค้า (Claim Analysis Report)
---

หลังจากที่เรานำข้อมูลจาก 3 แหล่ง (Orders, Deliveries, Claims) มารวมเป็นตารางเดียวแล้ว ขั้นตอนสุดท้ายคือการ **"บีบอัด"** ข้อมูลให้กลายเป็นตัวเลขสรุปครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Data Aggregation** | ดาต้า แอ็ก-เกร-เก-ชัน | **การรวบรวมข้อมูล** เพื่อหาผลสรุป (เช่น ยอดรวม, จำนวน) |
| **Revenue at Risk** | เร-เว-นิว แอท ริสค์ | **รายได้ที่มีความเสี่ยง** (ในที่นี้คือยอดเงินของสินค้าที่ถูกเคลม) |
| **Frequency Analysis** | ฟรี-เควน-ซี อะ-แน-ลิ-ซิส | **การวิเคราะห์ความถี่** (ดูว่าอะไรเกิดขึ้นบ่อยที่สุด) |
| **Grand Total** | แกรนด์ โท-ทัล | **ยอดรวมสุทธิ** ทั้งหมด |

### 💡 สิ่งที่ป๋าจะได้ทบทวน (Review)
1. การใช้ `groupby` เพื่อจัดกลุ่มตามชื่อสินค้า
2. การใช้ `count()` เพื่อดูจำนวนรายการ
3. การใช้ `sum()` เพื่อรวมมูลค่าความเสียหาย

In [12]:
# 1. กรองเอาเฉพาะรายการที่มีการเคลมจริง (ไม่ใช่ 'No Claim')
claims_only = df_final[df_final['CLAIM_STATUS'] != 'No Claim']

# 2. จัดกลุ่มตามประเภทสินค้า (PRODUCT) 
# และคำนวณ: นับจำนวนเคส (count) และ รวมราคา (sum)
claim_report = claims_only.groupby('PRODUCT')['PRICE'].agg(['count', 'sum'])

# 3. เปลี่ยนชื่อคอลัมน์ให้หัวหน้าอ่านง่าย
claim_report = claim_report.rename(columns={
    'count': 'จำนวนเคสที่เคลม',
    'sum': 'มูลค่าความเสียหายรวม'
})

# 4. ทบทวนบทที่ 3: ตกแต่งตารางให้สวยงาม
styled_report = claim_report.style.format({
    'มูลค่าความเสียหายรวม': '{:,.2f} บาท'
})

print("รายงานสรุปยอดเคลมแยกตามประเภทสินค้า:")
display(styled_report)

รายงานสรุปยอดเคลมแยกตามประเภทสินค้า:


,จำนวนเคสที่เคลม,มูลค่าความเสียหายรวม
PRODUCT,,
Router WiFi 6,2,"5,800.00 บาท"


## ✅ สรุปการเรียนรู้ของป๋า (Course Completion)
---

ยินดีด้วยครับป๋า! ตอนนี้ป๋าสามารถจัดการข้อมูล ISP และ Supply Chain ได้เหมือนมือโปรแล้ว:

1. **Path 1 (Foundation):** ป๋าเปิดไฟล์หลายชีทและคุม Type ข้อมูลได้แม่นยำ
2. **Path 2 (Calculation):** ป๋าคำนวณวันที่เหลือและสร้าง Logic `np.where` ตัดสินใจแทนคนได้
3. **Path 3 (Aggregation):** ป๋า Merge ตารางที่แยกกันอยู่ และสรุปผลออกมาเป็นรายงานได้

> **Note จากจุกทอง:** ข้อมูลแสนแถว หรือ 20 คอลัมน์ ก็ใช้วิธีเดียวกันนี้ครับ แค่ป๋าเลือกเฉพาะคอลัมน์ที่ใช้ (usecols) และหมั่นเช็กค่าว่าง (isna) ข้อมูลจะเยอะแค่ไหนป๋าก็เอาอยู่แน่นอน!